
# HDDM analysis of the 20Hz Data - Go-NoGo-Change task
Author: Julius Kricheldorff Affiliation: Carl von Ossietzky University Date: 09.05.2022

Import Neccessary packages:

In [23]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import hddm
import patsy
print(hddm.__version__)
os.chdir(r"C:\Users\doex9445\Dateien\Julius\20Hz\Data\Extracted")
os.getcwd()

0.8.0


'C:\\Users\\doex9445\\Dateien\\Julius\\20Hz\\Data\\Extracted'

Next we want to load the data. Because we want to analyze a GoNoGo task, our target (response) is not accuracy, but Go and NoGo trials. Thus we have to recode our dataset. Further we will include one variable with the stimulation conditions, one variable with the cued condition (cued Go or cued no Go) and see how these conditions affect response threshold. I think it might also be smart to distinguish cued NoGo Go and cued NoGo NoGo? i will have to think about the nested structure some more. First load the data:

In [15]:
data = hddm.load_csv('GoNoGo.csv')
# because we perform at least two model analyses - the first thing we want to see is how the Go trials and NoGo Cued trials differ by stimulation condition. For this analysis we ignore the NoGotrials and only at the Go trials

data_Go = data
# create a new column from existing column
data_Go['StopTrl'] = data_Go["GoNoGo"].apply(lambda x: 1 if "Stop" in x else 0)
data_Go['change_con'] = data_Go["GoNoGo"].apply(lambda x: 1 if "NoGo" in x else 0)
# filter Stop Trials
data_Go = data_Go[(data_Go.StopTrl != 1)]
# and rename a couple of columns
data_Go = data_Go.rename(columns={"RT":"rt", "Part_nr":"subj_idx", "Correct_Response": "response"})
data_Go.head(10)



,Trial_Nr,Required_Response,nrbeding,response,rt,Stimulus_Switch,Response_Switch,Post_Error,eingfehler,subj_idx,run_nr,UPDRS,Stim,Stim_verb,GoNoGo,StopTrl,change_con
1,2,right,19,1,0.984375,1,0,0,0,1,1,32,1,130Hz,NoGo - Go,0,1
2,3,right,18,1,0.484375,0,0,0,0,1,1,32,1,130Hz,NoGo - Go,0,1
3,4,right,19,1,0.656250,1,1,0,0,1,1,32,1,130Hz,Go,0,0
4,5,left,19,1,0.640625,1,0,0,0,1,1,32,1,130Hz,Go,0,0
5,6,right,17,1,0.640625,1,0,0,0,1,1,32,1,130Hz,NoGo - Go,0,1
6,7,right,18,1,0.906250,1,1,0,0,1,1,32,1,130Hz,Go,0,0
7,8,right,17,1,0.734375,0,0,0,0,1,1,32,1,130Hz,Go,0,0
9,10,right,16,1,0.546875,1,0,1,0,1,1,32,1,130Hz,NoGo - Go,0,1
11,12,left,18,1,0.515625,1,1,0,0,1,1,32,1,130Hz,Go,0,0
12,13,right,16,1,0.718750,1,0,0,0,1,1,32,1,130Hz,Go,0,0


Fit a first simple model:

In [16]:
md_srt_stim = hddm.HDDMRegressor(data_Go, 
                                 "a ~ Stim_verb:change_con", # threshold depends on Stimulation condition and whether Go trials were cued
                                p_outlier = 0.05) # we say that 5% of our trials are outlier trials)
# find a good starting value
md_srt_stim.find_starting_values()
#get samples and discard a couple as burn ins
md_srt_stim.sample(2000, burn = 100)

Adding these covariates:
['a_Intercept', 'a_Stim_verb[130Hz]:change_con', 'a_Stim_verb[20Hz]:change_con', 'a_Stim_verb[OFF]:change_con']


C:\Users\doex9445\AppData\Local\miniconda3\envs\DDM_space\lib\site-packages\scipy\optimize\optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----------------100%-----------------] 2000 of 2000 complete in 638.6 sec

And summarize the posterior distirbution:

In [34]:
#patsy.dmatrix("0 + Stim_verb:change_con", data_Go, )
patsy.dmatrix("0 + Stim + change_con + Stim:change_con", data_Go)
#stats = md_srt_general.gen_stats()
#stats = md_srt_general.gen_stats()
#stats[stats.index.isin(['a', 'v', 't'])]
#md_srt_general.plot_posterior_predictive(figsize=(14,10))


DesignMatrix with shape (3048, 3)
  Stim  change_con  Stim:change_con
     1           1                1
     1           1                1
     1           0                0
     1           0                0
     1           1                1
     1           0                0
     1           0                0
     1           1                1
     1           0                0
     1           0                0
     1           1                1
     1           0                0
     1           0                0
     1           0                0
     1           0                0
     1           0                0
     1           0                0
     1           1                1
     1           1                1
     1           0                0
     1           0                0
     1           1                1
     1           1                1
     1           1                1
     1           0                0
     1           1            

Now let us try the model of our choice, were the decision threshold varies as a function of stimulus condition and change warning.

Next we want to load the data. Because we want to analyze a GoNoGo task, our target (response) is not accuracy, but Go and NoGo trials. Thus we have to recode our dataset. Further we will include one variable with the stimulation conditions, one variable with the cued condition (cued Go or cued no Go) and see how these conditions affect response threshold. I think it might also be smart to distinguish cued NoGo Go and cued NoGo NoGo? i will have to think about the nested structure some more. First load the data:

In [ ]:

# create a new column from existing column
data_NoGo['response'] = data_NoGo["GoNoGo"].apply(lambda x: 1 if "Stop" in x else 0)
data_NoGo['change_con'] = data_NoGo["GoNoGo"].apply(lambda x: 1 if "NoGo" in x else 0)
# and rename a couple of columns
data_NoGo = data_NoGo.rename(columns={"RT":"rt", "Part_nr":"subj_idx"})
data_NoGo.head(10)

#Fit a first simple model:
md_srt_general = hddm.HDDM(data_NoGo,
                          p_outlier = 0.05) # we say that 5% of our trials are outlier trials)
# find a good starting value
md_srt_general.find_starting_values()
#get samples and discard a couple as burn ins
md_srt_general.sample(2000, burn = 100)

And summarize the posterior distirbution:


In [ ]:
stats = md_srt_general.gen_stats()
stats[stats.index.isin(['a', 'v', 't'])]
md_srt_general.plot_posterior_predictive(figsize=(14,10))


Now let us try the model of our choice, were the decision threshold varies as a function of stimulus condition and change warning.
